# Project-2

This is project-2 which will get you familiar with Structure from Motion and a rudimentary stereo SLAM pipeline. This notebook is not meant for coding, refer to the notebooks in `questions/` for more information.

**Team Name**: Sentinels

**Team Members/Rollnumbers**: 2019101054 2019111002

### Submission Instructions:
1. Zip the entire folder, not just this notebook, and submit on Moodle. 
2. Be sure to solve all questions that are in this notebook

This assignment will take around 10-12 hours, so be sure to split the work equally. A recommendation to split the work would be to have one person start with stereo, the other with bundle adjustment, and then both switch to completing the PnP and ICP parts. This is because question-1 and 2 are separate.

## Question 1: Stereo, PnP, and ICP

Time: ~6-7 Hours

This question has been designed to test your understanding of the concepts that create a SLAM system, from creating a front-end using Stereo Matching to a backend where you optimise to get your odometry to stitch the pointclouds obtained from the front end to build a map of the environment.

The dataset has been provided in `data/`. `img2` and `img3` correspond to the left and right camera respectively. `poses.txt` has the pose information for all frames. `calib.txt` has camera calibration information..

## Question 2: Bundle Adjustment

Time: ~2-3 Hours

This question will walk you through a simple implementation of Bundle Adjustment and run it on a a small dataset. Most of the weightage for this is in the theory part, where you have to explain the method.

# Question 1: Reading

1. Read the paper 'Building Rome in a Day' and briefly write about the fundamental idea behind the problem and solution. No need to be verbose, just write about the challenge with the task and how the pipeline is implemented (do not include details about performance/parallelization).

2. How is this task different from a SLAM problem?

Answer1. 
The fundamental idea of the problem beign proposed is that it is uncalibrated , unstructured , large and fast image matching and 3d reconstruction of a large dataset of images and  found by searching for Rome city on Internet sharing photo sites. The images are uncalibrated and without any gps data ,just matching similar images and construct 3d map with the help of matched images. Also the results are generated within a day in fast and effiecient manner. 
Results demonstrate that it is now possible to reconstruct city-scale image collections with more than a hundred thousand images in less than a day.
- Firstly they used SIFT for the extraction of image features
- Used Nearest-neighbour search for getting the correspondences for pair of images with those extracted features.
- Then a graph was constructed having images as vertices and there was an edge for similar images only , for this they randomly proposed an edge between vertices and only kept those which satisfy the similarity threshold.
- The previous portion is based on entire picture similarity, in which they treated images as documents and mapped characteristics to 'visual words'.
- Then they employed the TF-IDF-based document retrieval system to find the best/most comparable k1 + k2 photos to the current image under consideration. 
- Following that, they verified just a subset of the best k1 suggested edges.
- Further imporvement to this was added by decreasing the no. of connected components and verifying edges from remaining k2 suggestions
- At last the query expansion was applied where all vertices by distance of 2 edges are connected by direct edge and new edge is verified before insertion.
- Now the track of features was generated 
- For making the above mentioned change , they create a graph with all of the features from all of the photos as vertices and connections connecting matching/similar features.(connected corresponds to tracks for specific 3d points)
- Then they selected a subset of images to eliminate duplicate images in order to minimise the number of images for quicker processing in SFM without sacrificing accuracy.
- To acquire the 3D points and postures, they use custom SFM with bundle adjustment and the LM optimizer.
- At last they cluster the pictures and employ a multi-view stereo algorithm to enhance estimate of the sparse 3D points derived from SFM.

Answer2.
SLAM uses images or poses from the same sensor but in the mentioned case, we had images from different sensors whose calibrations are also not known.
In comparison to SLAM, the data is quite  scattered in above case.
The environment that we talk about in SLAM is small as compared to the above mentioned case.

# Theory

## 1. SfM pipeline (`6 mark`)

To get the context of below questions, take a look at the code above: The same questions have been asked at different places above as comments in the code.

1. `0.5 mark` **Basics** - How do we know this (`camera_ind`) information in practical setting? In other words, how do we know observations in `points_2d` belong to which camera. Explain. 
    - Ans-1 - Basics: In practical setting  we can correlate each image with a camera pose. As a result, there is a correspondence between the picture and the camera pose. We may extend this correspondence to the image coordinates because the points recorded in points 2d are from the image. Typically, these images are obtained through a feature extractor such as SIFT.



    
    
2. `0.5 mark` **Basics** - How do we know this (`point_ind`) information in practical setting?  In other words, how do we know observations in `points_2d` belong to which 3D point. Explain.
    - Ans-2 - Basics: In practical settings, we know that images in different perspectives are captured at different positions. We can use SIFT/SURF or ORB to determine corresponding points between two photos using this information. After this  8-point algorithm estimates the Fundamental Matrix, from which we may obtain the Essential Matrix and decompose it into R,t, and K. To obtain 3D coordinates, we can now triangulate or backproject points to the world. As a result, we now have our 2D-3D correspondences.


    
3. `0.5 mark` **Transformations** - `rotate()` function: Why do we use the rodriquez formula? How is this representation different from the standard 3x3 Rotation matrix, why do we use this instead?
    - Ans-3 - Transformations: Rodrigue's formula defines rotation of a point $P$ as $P$ being rotated by an angle $theta$ about a fixed axis $n$. 
    This is distinct from our usual Rotation matrix, in which we convert a point by multiplying it by a 3x3 rotation matrix (R). 
    So, in the instance of Rodrigue, we don't require a 3x3 matrix
    We only need minimum three parameters : two for the rotation vector  and one for theta. (the third can be computed from the other two because this vector is a unit vector).
    We do this because we have a camera params matrix (3x3) with 9 parameters in this example (including the 3 parameters required for rotation by rodrigues). 
    We were given a 3x4 projection matrix with a total of 12 parameters in our lesson (this is because it contains R,t and K). 
    So in all, we have $9*n cameras$ parameters to estimate in this scenario, but in the case of a 3x3 rotation matrix, we have $12*n cameras$ parameters to estimate.
    
    $$\mathbf{R}=\cos \theta \mathbf{I}+(1-\cos \theta) \mathbf{n n}^{\mathrm{T}}+\sin \theta \mathbf{n}^{\wedge}$$

    
4. `0.5 mark` **Transformations** - `project()` function: In the `project()` function, would it make any difference if I do translate first, then rotate? Why/why not?
    - Ans-4 - Transformations: Yes, there could be a distinction. Translation precedes rotation, implying that we move to a different origin first and then rotate about that origin. When we translate after rotating, we are also moving the rotation centre.
    If the translation vector is in the camera frame, the point must first be rotated to the camera frame before being added to the translation vector; otherwise, the translation vector and the point would be in different frames.


        
        
5. `0.5 mark` **Jacobian** - `bundle_adjustment_sparsity()` function: m above is not "M*N" (*2) unlike our lecture notes. Why is that so?
    - Ans-5 - Jacobian : As every row in Jacobian corresponds to a residue, we have as many rows as no. of residuals and that is equal to the total no. of 2d points we have as the loss function measures the diff between reporjected 2d points calc from estimated 3d points and estimated parameters and obs 2d points from image.
    Now because each 2d point contributes 2 coordinates and not one i.e x and y , so we have twice as many residuals as the number of 2d points.
    And it is different from class because in class we have assumed each N point is visible in M images so it leaves us with MN 2d points.Here however we have camera_indices.size 2d points as each element in camera_indices maps 2d point to image so twice as many rows or residuals in jacobian.
    
6. `2 mark` **Jacobian & Parameters** - `bundle_adjustment_sparsity()` function: 
    1.  Why are we doing `n_cameras * 9` here instead of `n_cameras * 12`? Recollect: Every individual motion Jacobian was (1*)12 in our lecture notes. 
        - Ans 6.1: Jacobian & Parameters: We used the homogeneous transformation matrix of dimensions 3x4 in the lecture, which gave us 12 unknowns for each camera. This matrix combines the intrinsics and the extrinsics (rotation and translation) into a single matrix, and we just optimise for it. 
        In this case, we segregated the various parameters by utilising distinct equations to describe the individual parts. 
        We employed one equation for the extrinsics, which included three rotation parameters and three translation parameters, and one equation for the intrinsics, which included three parameters (1 focal length + two radial distortion). 
        We optimised for each of these factors as a separate entity, resulting in just 9 unknowns per camera.   
        
    2. Ignoring the scale parameters, what was the number of unknown parameters in our lecture notes in terms of `n_cameras` and `n_points`? What is it here in the code? Is it different? If so, what is and why? [Link of notes](https://www.notion.so/Stereo-Structure-from-Motion-9fdd81e4194f4803ac9ba7552df56470).
        - Ans 6.2: Jacobian and Parameters: We had `n points * 3 + n cameras * 12` unknowns in the lecture and   we have `n cameras * 9 + n points * 3` unknowns in the code.
        we have `n cameras * 9 + n points * 3` unknowns in the code. The primary difference here is the quantity of unknowns associated with the poses/camera frames. 
        In the lecture, we avoided optimising over the SO(3) manifold by optimising over the projection matrix of dimensions 3x4 and only required to explicitly estimate the 12 projection parameters, which comprised both extrinsics and intrinsics in a single matrix. In contrast, we model the extrinsics and intrinsics independently in this case, employing a rotation vector (3), a translation vector (3), and three extra parameters of focal length (1) and radial distortion (2), totaling nine unknowns for each view of the camera.         
            
            
7. `6 mark` **Sparsity, Residual Vector & Jacobian** - `bundle_adjustment_sparsity()` function: Explain what you understand from above 6 lines of code by coding a simple toy example yourself to illustrate how it is different from what you've learnt in class. ([Coding toy example + elaborating in words]- both are compulsory.) For the toy example, you can take something like 3 points all seen from 3 cameras. (You don't actually have to code much, just need to call the existing function) Write that toy example after this cell
    - Ans 6 - Sparsity, Residual Vector & Jacobian: 
- This code essentially initialises the sparse Jacobian by designating non-zero items as 1 and leaving the remainder as 0.
- For each of the two rows of each observation/2D image point, the 9 columns of the corresponding camera parameters and the 3 columns of the corresponding 3D point are set to one.
- Because they have no effect on the observation, the remaining values/columns are set to 0.  The next for loop sets 3 cols corresponding to the world point in the ith observation to 1 for all i. Other elements remain 0 and form a sparse structure.
- Each column in the Jacobian represents an unknown parameter being optimised, whereas each row represents an observation/equation. Because each observation/2D point has two equations, one for the x and one for the y, we have 2 * num obs rows. Because we have (n cameras * 9 + n points * 3) unknowns, the Jacobian has (n cameras * 9 + n points * 3) columns.
- The distinction is that in our lecture notes, we used M for the number of photos and N for the number of points in each image, and we had MN 2D points, each with two coordinates, resulting in 2MN residuals for the Jacobian rows. 
- m has a distinct size since it contains information about all of the observation made collectively (all 2D points), i.e., 'camera indices.size' 2D points.
- In lectures, residuals were arranged first by points and then by pictures, but here they appear in no particular sequence, which has no effect if order is maintained along the pipeline.


In [1]:
import numpy as np 
from scipy.sparse import lil_matrix

n_camera = 3
n_point = 3
camera_iter=np.arange(n_camera)
camera_idx = np.array([camera_iter]*n_point)
camera_idx=camera_idx.T.flatten()
point_iter=np.arange(n_point)
point_idx = np.array([point_iter]*n_camera)
point_idx=point_idx.flatten()

print("camera_idx = ",camera_idx,end="\n")
print("point_idx = ", point_idx,end="\n")

def bundle_adjustment_sparsity(n_cameras, n_points, camera_indices, point_indices):
    m = 2* camera_indices.size 
    n = n_cameras * 9 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    camera_indices = np.sort(camera_indices)
    point_indices = np.sort(point_indices)
    
    i = np.arange(camera_indices.size)
    
    for s in range(9):
        A[2 * i, camera_indices * 9 + s] = 1
        A[2 * i + 1, camera_indices * 9 + s] = 1
    for s in range(3):
        A[2 * i, n_cameras * 9 + point_indices * 3 + s] = 1
        A[2 * i + 1, n_cameras * 9 + point_indices * 3 + s] = 1
    return A

print("Sparse Jacobian is :")
J=bundle_adjustment_sparsity(n_camera, n_point, camera_idx, point_idx)
print(J)


camera_idx =  [0 0 0 1 1 1 2 2 2]
point_idx =  [0 1 2 0 1 2 0 1 2]
Sparse Jacobian is :
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 27)	1
  (0, 28)	1
  (0, 29)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 27)	1
  (1, 28)	1
  (1, 29)	1
  (2, 0)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	1
  (2, 4)	1
  (2, 5)	1
  (2, 6)	1
  (2, 7)	1
  (2, 8)	1
  (2, 27)	1
  (2, 28)	1
  (2, 29)	1
  (3, 0)	1
  (3, 1)	1
  (3, 2)	1
  (3, 3)	1
  (3, 4)	1
  (3, 5)	1
  (3, 6)	1
  (3, 7)	1
  (3, 8)	1
  (3, 27)	1
  (3, 28)	1
  (3, 29)	1
  (4, 0)	1
  (4, 1)	1
  (4, 2)	1
  (4, 3)	1
  (4, 4)	1
  (4, 5)	1
  (4, 6)	1
  (4, 7)	1
  (4, 8)	1
  (4, 27)	1
  (4, 28)	1
  (4, 29)	1
  (5, 0)	1
  (5, 1)	1
  (5, 2)	1
  (5, 3)	1
  (5, 4)	1
  (5, 5)	1
  (5, 6)	1
  (5, 7)	1
  (5, 8)	1
  (5, 27)	1
  (5, 28)	1
  (5, 29)	1
  (6, 9)	1
  (6, 10)	1
  (6, 11)	1
  (6, 12)	1
  (6, 13)	1
  (6, 14)	1
  (6, 15)	1
  (6, 16)	1
  (6, 1

## 2. Initializing R,t and 3D points for SfM given 2 images (`4 mark`)

Using OpenCV functions, mention how you would initialize R,t (poses) and 3D points for SfM given 2 images and K matrix. You don't need to implement it, just mention function names with input/output arguments clearly and briefly explain what they do (You don't need to give detailed answers).

*Answer2.* 
Given: 2 images and K matrix 
1. **First we do feature matching** 
For doing this we will create an instance of ORB detector class (in openCV ofc) with the help of `cv2.ORB_create()`. 
Then we use `detectAndCompute()` as a fuction to detector for extracting ORB features which constitues descriptors as well as keypoints on both images.(we pass none for showing we are not passing the features that are predefined) 

    ``` orb=cv2.ORB_create() ``` 
    ``` Keypoint1,descriptor1=orb.detectAndCompute(image1,None) ``` 
    ``` Keypoint2,descriptor2=orb.detectAndCompute(image2,None) ``` 

2. After this , we need to do **matching of the keypoints from the two images.** 
For doing this we will create brute-force matcher with the help of `cv2.BFMatcher()` 
Then with the help of ORB descriptors we will perform KNN matching with the help of `knnMatch()` 
And now we have the matching features from both the images. 

    ```brute_force=cv2.BFMatcher() ```
    ```Matches=brute_force.knnMatch(descriptor1,descriptor2,k=2) ```

    [we can also use ```brute_force.match(descriptor1,descriptor2)```]

3. From the above two steps, we have matched features from both the images. Now with the help of these matches, we find fundamental matrix with the help of cv2.finfFundamentalMat()

    [ Note: we can also use normalized 8 point algo ]

4. After we have `F` and since we know `K` we can now have the essential matrix as $ E= K^TFK $ .


Note: we can also combine points 3 and 4 in the following ways:

We can get Essential matrix directly with the help of `E=cv2.findEssentialMatrix(points1,points2,K)` Here, we input the corresponding points in the two images (points1,points2) and also the K matrix.
Now we have `E`, we can decompose it into R,t as 
`output=cv2.recoverPose(E,points1,points2,K) ` . Here we input `E` , matches `(points1,points2)` and calibration matrix `K` and gives `R` which is output[1] and `t` from output[2]. 

Then we use triangulation to initialize 3D poses and use `points_3D=cv2.triangulatePoints([points1,points2,output])` and so we get the 3D points as output !